### Evaluación de muestros modelos dataset Calidad Aire

Librerias


In [24]:
# Tratamiento de datos
import pandas as pd
import numpy as np
# Almacenar en caché los resultados de funciones en el disco
import joblib
import os
# Gestion de librerias
from importlib import reload
# Matemáticas y estadísticas
import math
# Preparación de datos
from sklearn.neighbors import LocalOutlierFactor
#Separar los datos entrenamiento y prueba
from sklearn.model_selection import train_test_split
#Escalar Variables
from sklearn.preprocessing import MinMaxScaler
#Evaluación del modelo
from sklearn import metrics
#from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
#from sklearn.metrics import roc_auc_score
#from sklearn.metrics import roc_curve
#Creación de modelo
from sklearn import svm
#configuracion de hiperparámetros
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

### Cargamos el dataset calidad de aire

In [25]:
#Carga del dataset
dataCalidadAire =pd.read_csv("data/stage/dataSetLimpio4.csv")
dataCalidadAire.head(3)

,anio,mes,dia,hora,pm25,festivo,temperatura,humedad,presion,p1,...,dia_semana_Inicio Semana,dia_semana_Media semana,estacion_Estacion Aranjuez,estacion_Estacion Belen,estacion_Estacion Caldas,estacion_Estacion Itagui,franja_horaria_Madrugada,franja_horaria_Mañana,franja_horaria_Noche,franja_horaria_Tarde
0,2024,1,1,0,16,1,20,84,0,0,...,1,0,0,0,0,1,1,0,0,0
1,2024,1,1,2,47,1,18,82,0,0,...,1,0,0,0,0,1,1,0,0,0
2,2024,1,1,6,39,1,17,84,0,0,...,1,0,0,0,0,1,0,1,0,0


In [26]:
dataCalidadAire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19447 entries, 0 to 19446
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   anio                        19447 non-null  int64
 1   mes                         19447 non-null  int64
 2   dia                         19447 non-null  int64
 3   hora                        19447 non-null  int64
 4   pm25                        19447 non-null  int64
 5   festivo                     19447 non-null  int64
 6   temperatura                 19447 non-null  int64
 7   humedad                     19447 non-null  int64
 8   presion                     19447 non-null  int64
 9   p1                          19447 non-null  int64
 10  velocidad_prom              19447 non-null  int64
 11  velocidad_max               19447 non-null  int64
 12  direccion_prom              19447 non-null  int64
 13  direccion_max               19447 non-null  int64
 14  dia_se

#### Definimos nuestro X y Y

In [27]:
#Se establece las variables de entrada 'X' y la variable de salida 'y'

X = dataCalidadAire.drop(columns ='pm25')
y = dataCalidadAire['pm25']

In [28]:
#Escalar Variables númericas
columnas = X.columns.to_list()
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Cargar el scaler
scaler = joblib.load("data/stage/estandarizacionDataSet4.pkl")

# Se escalan los valores del dataset entrenamiento y prueba de las columnas numéricas
X[columnas] = scaler.transform(X[columnas])

X[columnas].head(2)

,anio,mes,dia,hora,festivo,temperatura,humedad,presion,p1,velocidad_prom,...,dia_semana_Inicio Semana,dia_semana_Media semana,estacion_Estacion Aranjuez,estacion_Estacion Belen,estacion_Estacion Caldas,estacion_Estacion Itagui,franja_horaria_Madrugada,franja_horaria_Mañana,franja_horaria_Noche,franja_horaria_Tarde
0,0.0000,0.0000,0.0000,0.0000,1.0000,0.3636,0.8256,0.0000,0.0000,0.2000,...,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000
1,0.0000,0.0000,0.0000,0.0870,1.0000,0.2727,0.8023,0.0000,0.0000,0.4000,...,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000


## Evaluación del modelo

#### validaciones finales

In [29]:
X.shape

(19447, 24)

In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19447 entries, 0 to 19446
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   anio                        19447 non-null  float64
 1   mes                         19447 non-null  float64
 2   dia                         19447 non-null  float64
 3   hora                        19447 non-null  float64
 4   festivo                     19447 non-null  float64
 5   temperatura                 19447 non-null  float64
 6   humedad                     19447 non-null  float64
 7   presion                     19447 non-null  float64
 8   p1                          19447 non-null  float64
 9   velocidad_prom              19447 non-null  float64
 10  velocidad_max               19447 non-null  float64
 11  direccion_prom              19447 non-null  float64
 12  direccion_max               19447 non-null  float64
 13  dia_semana_Fin de Semana    194

Ruta donde se encunetra nuestros modelos de regresion PM25

In [31]:
# Ruta donde se encuentran los modelos
ruta_modelos = 'data/analytics/modelosPM25'

In [32]:
# Lista para almacenar los modelos cargados
modelos_cargados = []
dr = pd.DataFrame()

# Itera sobre los archivos en la ruta
for archivo_modelo in os.listdir(ruta_modelos):
    # Comprueba si el archivo es un modelo 
    if archivo_modelo.endswith('.pkl'):
        print(f"Este es mi modelo {archivo_modelo}")
        ruta_completa = os.path.join(ruta_modelos, archivo_modelo)
        # Carga el modelo
        modelo_cargado = joblib.load(ruta_completa)
        # Agrega el modelo a la lista
        modelos_cargados.append(archivo_modelo)
        print (f"mi lista modelos cargados esta {modelos_cargados}")
        # Realizar predicciones
        y_pred = modelo_cargado.predict(X)
        print(f"Mi modelo {archivo_modelo} predice {y_pred}")
        # Adicionar una nueva fila como un DataFrame
        nuevoModelo = pd.DataFrame({
            'model': [archivo_modelo],
            'Descripcion': [modelo_cargado],
            'r2_score': [metrics.r2_score(y, y_pred)],
            'mae': [metrics.mean_absolute_error(y, y_pred)],
            'rmse': [np.sqrt(metrics.mean_squared_error(y, y_pred))]
        })
        dr = pd.concat([dr, nuevoModelo], ignore_index=True)

dr

Este es mi modelo modeloRandomForestPM25.pkl
mi lista modelos cargados esta ['modeloRandomForestPM25.pkl']
Mi modelo modeloRandomForestPM25.pkl predice [21.46322081 23.49651218 25.7381408  ... 25.79419523 25.79419523
 26.32333495]
Este es mi modelo modeloRegresionLinealPm25.pkl
mi lista modelos cargados esta ['modeloRandomForestPM25.pkl', 'modeloRegresionLinealPm25.pkl']
Mi modelo modeloRegresionLinealPm25.pkl predice 0       122.3522
1       122.4593
2       127.1150
3       127.1370
4       127.1424
          ...   
19442   -43.7363
19443   -43.7370
19444   -43.7442
19445   -43.7443
19446   -43.7513
Length: 19447, dtype: float64
Este es mi modelo SVR_CV_varios_cal_aire_pm25.pkl
mi lista modelos cargados esta ['modeloRandomForestPM25.pkl', 'modeloRegresionLinealPm25.pkl', 'SVR_CV_varios_cal_aire_pm25.pkl']
Mi modelo SVR_CV_varios_cal_aire_pm25.pkl predice [16.11006369 16.37922525 22.75015431 ... 28.1386797  28.25002968
 31.10031868]


,model,Descripcion,r2_score,mae,rmse
0,modeloRandomForestPM25.pkl,"(DecisionTreeRegressor(max_depth=8, max_featur...",0.4869,5.7326,7.2699
1,modeloRegresionLinealPm25.pkl,<statsmodels.regression.linear_model.Regressio...,-53.2685,71.8625,74.7637
2,SVR_CV_varios_cal_aire_pm25.pkl,"SVR(C=1, gamma=1, kernel='poly')",0.4095,5.9235,7.7991


In [33]:
# Top n de los mejores modelos
dr.sort_values(by='r2_score', ascending=False)

,model,Descripcion,r2_score,mae,rmse
0,modeloRandomForestPM25.pkl,"(DecisionTreeRegressor(max_depth=8, max_featur...",0.4869,5.7326,7.2699
2,SVR_CV_varios_cal_aire_pm25.pkl,"SVR(C=1, gamma=1, kernel='poly')",0.4095,5.9235,7.7991
1,modeloRegresionLinealPm25.pkl,<statsmodels.regression.linear_model.Regressio...,-53.2685,71.8625,74.7637
